## <div align="center"> UJIAN TENGAH SEMESTER IF540-L MACHINE LEARNING </div>
## <div align="center"> Semester Gasal 2024/2025 </div>
## <div align="center"> Pengembangan Model Prediksi Tipe Curah Hujan dengan Analisis Regresi pada Data Cuaca Historis </div>

---
### Kelompok - [isi nomor kelompok]

#### Anggota Kelompok : 
1. Brightly Virya
2. Batara Hotdo Horas Simbolon
3. Daniel
4. Christopher Alvaro
...

---

### Dataset yang digunakan untuk projek:

1. weatherHistory.csv – sumber : https://www.kaggle.com/datasets/muthuj7/weather-dataset

### Instaling dependencies
* don't need to run this code if you already have all the dependencies

In [1]:
%pip install pandas
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import all the libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

### Pre-processing
* Memisahkan Train data dan Test data
* 10% data paling baru pada weather history digunakan untuk test data menguji keakuratan model.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Load the dataset
file_path = 'weatherHistory.csv' 
df = pd.read_csv(file_path)

# Convert 'Formatted Date' to datetime
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'])

# Sort data by 'Formatted Date' to ensure it's in chronological order
df = df.sort_values(by='Formatted Date')

# Set 10% of the latest data as test data
test_size = 0.1
n_test = int(len(df) * test_size)

# Split the data into train and test sets
train_data = df.iloc[:-n_test]
test_data = df.iloc[-n_test:]

# Check the shape of train and test sets
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

# Save the train and test data to separate CSV files
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

Train data shape: (86808, 12)
Test data shape: (9645, 12)


### Load train data

In [4]:
df = pd.read_csv('train_data.csv')
df

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-01-01 00:00:00+01:00,Partly Cloudy,rain,0.577778,-4.050000,0.89,17.1143,140.0,9.9820,0.0,1016.66,Mostly cloudy throughout the day.
1,2006-01-01 01:00:00+01:00,Mostly Cloudy,rain,1.161111,-3.238889,0.85,16.6152,139.0,9.9015,0.0,1016.15,Mostly cloudy throughout the day.
2,2006-01-01 02:00:00+01:00,Mostly Cloudy,rain,1.666667,-3.155556,0.82,20.2538,140.0,9.9015,0.0,1015.87,Mostly cloudy throughout the day.
3,2006-01-01 03:00:00+01:00,Overcast,rain,1.711111,-2.194444,0.82,14.4900,140.0,9.9015,0.0,1015.56,Mostly cloudy throughout the day.
4,2006-01-01 04:00:00+01:00,Mostly Cloudy,rain,1.183333,-2.744444,0.86,13.9426,134.0,9.9015,0.0,1014.98,Mostly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
86803,2015-11-25 22:00:00+01:00,Overcast,rain,2.688889,-0.827778,0.86,13.5884,20.0,8.4203,0.0,1007.94,Mostly cloudy throughout the day.
86804,2015-11-25 23:00:00+01:00,Overcast,rain,2.127778,-0.655556,0.91,9.5956,20.0,7.2128,0.0,1007.76,Mostly cloudy throughout the day.
86805,2015-11-26 00:00:00+01:00,Overcast,rain,2.127778,-1.000000,0.89,11.0607,11.0,6.2951,0.0,1007.48,Foggy in the morning.
86806,2015-11-26 01:00:00+01:00,Overcast,rain,1.688889,-1.388889,0.92,10.4650,2.0,5.5706,0.0,1007.16,Foggy in the morning.


### Another pre-process

In [5]:
# Handle missing values in 'Precip Type' column by filling with the most frequent value
precip_imputer = SimpleImputer(strategy='most_frequent')
df['Precip Type'] = precip_imputer.fit_transform(df[['Precip Type']])

# Drop unnecessary columns
# 'Formatted Date', 'Summary', and 'Daily Summary' are dropped as they are not suitable for logistic regression
# due to being non-numeric and unlikely to have a useful numerical transformation
df = df.drop(columns=['Formatted Date', 'Summary', 'Daily Summary'])

# Encode categorical data in 'Precip Type' column to numerical
label_encoder = LabelEncoder()
df['Precip Type'] = label_encoder.fit_transform(df['Precip Type'])

# Ensure all feature columns are numeric and handle any remaining non-numeric data
df = pd.get_dummies(df, drop_first=True)

# Split features and target variable
X = df.drop(columns=['Precip Type'])
y = df['Precip Type']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display pre-processed data
print("Features after preprocessing:", X[:5])
print("Target after preprocessing:", y[:5])

ValueError: 2

### Drop unnecessary columns
'Formatted Date', 'Summary', and 'Daily Summary' are dropped as they are not suitable for logistic regression due to being non-numeric and unlikely to have a useful numerical transformation


In [ ]:
df = df.drop(columns=['Formatted Date', 'Summary', 'Daily Summary'])
df

### Hasil kerja

In [ ]:
# Your codes are here (replace the following codes)
import time
time.sleep(10)

In [ ]:
!jupyter nbconvert --to html "./UTS2024_IF540L_E_Kelompok_4.ipynb" --output-dir="./"

### Next step:
* convert the generated html file to PDF using the online tool: https://www.sejda.com/html-to-pdf
* choose the following settings:
    * Page size: One long page
    * Page Orientation: auto
    * Use print stylesheet
* Submit your ipython notebook and PDF files

Markdown basics https://markdown-guide.readthedocs.io/en/latest/basics.html#